In [ ]:
import heapq

class PuzzleNode:
    """
    Represents a node in the search tree for the 8-puzzle problem.
    """
    def __init__(self, state, parent=None, move=None, g=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.g = g  # Cost from start node (depth)
        self.h = 0  # Heuristic cost
        self.f = 0  # Evaluation function f(n) = g(n) + h(n)

    def __lt__(self, other):
        # Comparison for the priority queue
        return self.f < other.f

    def __eq__(self, other):
        # Equality check for states
        return self.state == other.state


In [ ]:

def print_board(state):
    """Prints the 3x3 board."""
    for row in state:
        print(" ".join(map(str, row)))
    print()

def get_blank_position(state):
    """Finds the (row, col) of the blank tile (0)."""
    for r, row in enumerate(state):
        for c, val in enumerate(row):
            if val == 0:
                return r, c
    return None

def h1_misplaced_tiles(state, goal_state):
    """Heuristic 1: Counts the number of misplaced tiles."""
    misplaced = 0
    for r in range(3):
        for c in range(3):
            if state[r][c] != goal_state[r][c] and state[r][c] != 0:
                misplaced += 1
    return misplaced

def h2_manhattan_distance(state, goal_state):
    """Heuristic 2: Calculates the sum of Manhattan distances for all tiles."""
    distance = 0
    goal_positions = {}
    for r, row in enumerate(goal_state):
        for c, val in enumerate(row):
            goal_positions[val] = (r, c)
            
    for r in range(3):
        for c in range(3):
            val = state[r][c]
            if val != 0:
                goal_r, goal_c = goal_positions[val]
                distance += abs(r - goal_r) + abs(c - goal_c)
    return distance

def get_successors(node):
    """Generates all valid successor nodes."""
    successors = []
    r, c = get_blank_position(node.state)
    moves = {'UP': (-1, 0), 'DOWN': (1, 0), 'LEFT': (0, -1), 'RIGHT': (0, 1)}

    for move, (dr, dc) in moves.items():
        nr, nc = r + dr, c + dc
        if 0 <= nr < 3 and 0 <= nc < 3:
            new_state = [row[:] for row in node.state]
            new_state[r][c], new_state[nr][nc] = new_state[nr][nc], new_state[r][c]
            successors.append(PuzzleNode(new_state, parent=node, move=move, g=node.g + 1))
    return successors

def solve_puzzle(initial_state, goal_state, algorithm, heuristic_func):
    """
    Solves the 8-puzzle problem using the specified algorithm and heuristic.
    """
    start_node = PuzzleNode(initial_state)
    start_node.h = heuristic_func(start_node.state, goal_state)
    
    if algorithm == 'A*':
        start_node.f = start_node.g + start_node.h
    else: # Best-First Search
        start_node.f = start_node.h

    open_list = [start_node] # Priority queue
    closed_list = set()
    step = 0
    max_steps = 200 # Set the maximum number of steps

    print(f"--- Starting {algorithm} with {heuristic_func.__name__} ---")
    
    while open_list:
        step += 1
        
        # <<< --- ADDED STOP FEATURE --- >>>
        if step > max_steps:
            print(f"--- Step {step} ---")
            print(f"Search terminated: Exceeded maximum of {max_steps} steps.")
            return
        # <<< --- END OF ADDED FEATURE --- >>>

        current_node = heapq.heappop(open_list)
        
        # Convert list of lists to tuple of tuples to add to set
        state_tuple = tuple(map(tuple, current_node.state))

        if state_tuple in closed_list:
            continue
        
        closed_list.add(state_tuple)
        
        print(f"--- Step {step} ---")
        print(f"Expanding Node with f(n) = {current_node.f}, g(n) = {current_node.g}, h(n) = {current_node.h}")
        print_board(current_node.state)

        if current_node.state == goal_state:
            print("Goal Reached!")
            path = []
            node = current_node
            while node.parent is not None:
                path.append(node.move)
                node = node.parent
            path.reverse()
            print(f"Solution Path: {' -> '.join(path)}")
            print(f"Total steps (nodes expanded): {step}")
            print(f"Solution Depth (cost): {current_node.g}\n\n")
            return

        print("Generating Successors...")
        successors = get_successors(current_node)
        for successor in successors:
            successor_state_tuple = tuple(map(tuple, successor.state))
            if successor_state_tuple in closed_list:
                continue

            successor.h = heuristic_func(successor.state, goal_state)
            if algorithm == 'A*':
                successor.f = successor.g + successor.h
            else: # Best-First Search
                successor.f = successor.h
                
            heapq.heappush(open_list, successor)
            print(f"  -> Successor (move {successor.move}) with f={successor.f}, g={successor.g}, h={successor.h}")
            
        print("\nOpen List (Top 5 states by f-value):")
        for i, node in enumerate(heapq.nsmallest(5, open_list)):
            print(f"  {i+1}. f={node.f}, g={node.g}, h={node.h}")



In [ ]:

if __name__ == "__main__":

    initial_state = [
        [1, 2, 3],
        [0, 4, 5],
        [6, 7, 8]
    ]

    goal_state = [
        [1, 2, 3],
        [4, 0, 5],
        [6, 7, 8]
    ]

    print("Initial State:")
    print_board(initial_state)
    print("Goal State:")
    print_board(goal_state)


    print("select your preferered algorithm and heuristic (A* or BFS, h1 or h2): \n 1. A* with Misplaced Tiles\n 2. A* with Manhattan Distance\n 3. Best-First Search with Misplaced Tiles\n 4. Best-First Search with Manhattan Distance\n")
    switch = input("Enter your choice (1-4): ")
    # Based on the input, we can choose the algorithm and heuristic
    if switch == '1':
        solve_puzzle(initial_state, goal_state, 'A*', h1_misplaced_tiles)
    elif switch == '2':
        solve_puzzle(initial_state, goal_state, 'A*', h2_manhattan_distance)
    elif switch == '3':
        solve_puzzle(initial_state, goal_state, 'BFS', h1_misplaced_tiles)
    elif switch == '4':
        solve_puzzle(initial_state, goal_state, 'BFS', h2_manhattan_distance)
    # # Case 1: A* with Misplaced Tiles
    # solve_puzzle(initial_state, goal_state, 'A*', h1_misplaced_tiles)

    # # Case 2: A* with Manhattan Distance
    # solve_puzzle(initial_state, goal_state, 'A*', h2_manhattan_distance)
    
    # # Case 3: Best-First Search with Misplaced Tiles
    # solve_puzzle(initial_state, goal_state, 'BFS', h1_misplaced_tiles)
    
    # # Case 4: Best-First Search with Manhattan Distance
    # solve_puzzle(initial_state, goal_state, 'BFS', h2_manhattan_distance)